In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_parquet('data/runs.parquet')

In [ ]:
def dcg(data, k):
    ranking = data.Relevancy.astype(int).head(k).tolist()
    return ranking [0] + sum(list(map(lambda entry: entry[1]/np.log(1+entry[0]), enumerate(ranking[1:], start = 1))))

l = []
for K in [5,10,15,20]:
    scores = df.loc[:,['TREC','Topic','Run']]

    scores = scores.merge(
        df
        .sort_values('Rank', ascending = True)
        .groupby(['TREC','Topic','Run'])
        .apply(dcg, K)
        .reset_index()
        .rename({0:'DCG'},axis = 1),
        on = ['TREC','Topic','Run'],
        how = 'left'
    )
    scores = scores.merge(
        df
        .sort_values('Relevancy', ascending = False)
        .groupby(['TREC','Topic'])
        .apply(dcg, K)
        .reset_index()
        .rename({0:'IDCG'},axis = 1),
        on = ['TREC','Topic'],
        how = 'left'
    )
    scores = scores.merge(
        df
        .sort_values('Relevancy', ascending = True)
        .groupby(['TREC','Topic'])
        .apply(dcg, K)
        .reset_index()
        .rename({0:'WDCG'},axis = 1),
        on = ['TREC','Topic'],
        how = 'left'
    )
    scores['K'] = K
    l.append(scores)

scores = pd.concat(l)

def normalize(v,min_v,max_v):
    return np.float64(v-min_v)/np.float64(max_v-min_v)


scores['NDCG_org'] = scores.apply(lambda row: normalize(row['DCG'],0,row['IDCG']),axis=1)
scores['NDCG_min'] = scores.apply(lambda row: normalize(row['DCG'],row['WDCG'],row['IDCG']),axis=1)
scores['NDCG_0'] = scores.apply(lambda row: np.float64(row['WDCG'])/np.float64(row['IDCG']),axis=1)
scores.drop_duplicates().to_parquet('data/scores.parquet', compression = 'GZIP')